In [34]:
import random

class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()        

    def get_propaganda(self):
        if int(self.SUBJprop) <=3 :
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [x.propaganda for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)

In [35]:
import pandas as pd
import numpy as np
import nltk
import string
from string import digits

import nltk


reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))))



Total Rows:
16774
Total Positive:
3780
Total Negative:
12994


In [37]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
punct = string.punctuation

stopwords = list(STOP_WORDS)


nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens


In [ ]:
text_data_cleaning("    Hello how are you. Like this video")


In [38]:
from sklearn.model_selection import train_test_split

neg_prop = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))
pos_prop = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))

########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, random_state = 42 )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, random_state = 42 )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, random_state = 42 )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, random_state = 42 )

dev = neg_dev + pos_dev
random.shuffle(dev)

test = neg_test + pos_test
random.shuffle(test)
########################################################################################
print("Total Train:")
print(len(train))
print("Positive:")
print(len(pos_train))
print("Negative:")
print(len(neg_train))

print("\nTotal Development:")
print(len(dev))
print("Positive:")
print(len(pos_dev))
print("Negative:")
print(len(neg_dev))

print("\nTotal Test:")
print(len(test))
print("Positive:")
print(len(pos_test))
print("Negative:")
print(len(neg_test))

Total Train:
11741
Positive:
2646
Negative:
9095

Total Development:
2516
Positive:
567
Negative:
1949

Total Test:
2517
Positive:
567
Negative:
1950


In [39]:
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_sentence = train_container.get_sentence()   
train_propaganda = train_container.get_propaganda() 

development_sentence = [x.sentence for x in dev]
development_propaganda = [x.propaganda for x in dev]

test_sentence = [x.sentence for x in test]
test_propaganda = [x.propaganda for x in test]

print("Total rows after balance training dataset:")
print(len(train_sentence))

print("Positive Propaganda:")
print(train_propaganda.count(Propaganda.POSITIVE))

print("Negative Propaganda:")
print(train_propaganda.count(Propaganda.NEGATIVE))


Total rows after balance training dataset:
5292
Positive Propaganda:
2646
Negative Propaganda:
2646


In [40]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [41]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [65]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
class word2vec():
    def get_vec(self,x):
        train_x_vectors = []
        for x in train_sentence:
            train_x_vectors.append(nlp(x).vector)

            # yield sparse2full(docvec, len(self.id2word))   
# # Word2Vec
# train_x_vectors = []
# for x in train_sentence:
#      train_x_vectors.append(get_vec(x))

# word2vec = nlp(x).vector

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(train_sentence,train_propaganda)

In [66]:
clf = Pipeline([('word2vec',word2vec()), ('clf', classifier)])
clf.fit(train_sentence,train_propaganda)

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '<__main__.word2vec object at 0x7fc66fdd4640>' (type <class '__main__.word2vec'>) doesn't

In [51]:
y_pred = clf.predict(development_sentence)

print(classification_report(development_propaganda, y_pred))



ValueError: Expected 2D array, got 1D array instead:
array=['• On January 24, 2017, the CAO acquired the [r[replacement]erver from the control of the employees and transferred that server to the USCP.'
 'Over that same period Hispanic employment rose by 3.9 million, up a whopping 24.2%.'
 'as it happens that is not the case' ...
 'All presidents since Clinton have crossed countries off this list.'
 '“boy that is just the absolute lowest aiming possibility we could come up with do you believe in icons”'
 'They are afraid of the truth.'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.